## Collaborative Filtering vs. Content-based Filtering

In **Collaborative Filtering**, the algorithm recommends items to the users based on ratings of users who gave similar ratings as the same user.

In **content-based filtering** the algorithm recommends items to the user based on features of user and item to find a good match.

Now, we will continue to use, 

$r(i,j) = 1, \text{if user j has rated item i}$  
$y^{(i,j)} = \text{rating given by user j on item i}$

The key idea behind content-based filtering is that we are going to make a good use of features of the items and the user to make prediction on what the user might like. 

## In case of movie recommendation example,


We might have features of an user like,

- Age
- Gender
- Country
- Movies Watched
- Avg. given ratings per genre
We can come up with a user feature vector $X_u^{(j)}$

Also, We might have features of movies, 
- Year of release
- Genre/Genres
- Reviews
- Avg. ratings

We can come up with a movie feature vector $X_m^{(i)}$


## We are going to predict weather a movie $i$ will be a good match for the user $j$

### Intuition

Consider two vectors,

$V_u^{(j)}$ computed from $X_u^{(j)}$  
and  
$V_m^{(i)}$ computed from $X_m^{(i)}$  


These two vectors will capture the preference of the user and and the features of the movies.

For example,

$V_u^{(1)}$ can be an array of 30, with each element represents a floating point number which approximates the likeliness of a certain movie genre(user preferences).

Also, 
$V_m^{(i)}$ is also a array of size 30(need to be as of same size to compute dot product) which captures the association of the movie with particular genre.

### The problem is how can we approximate $V_u^{(j)}$ from $X_u^{(j)}$  and $V_m^{(i)}$  from $X_m^{(i)}$  


<img src="./screenshots/lerning-top-match.png" width="700">

## Content-based Filtering Using Neural Networks

We will use two neural networks to generate two vectors which when dot producted will give us our predictions.

The architecture will be:

<img src="./screenshots/NN-Architecture-CollaborativeFiltering.png" width="700">


## Cost Function



$$J=\sum_{(i,j): r(i,j)=1} (V_u^{(j)} \cdot V_m^{(i)} - y^{(i,j)})^2 + \text{NN Regularisation Term}$$

## Finiding the similar movies or similar items

$ \bmod(V_m^{(k)} - V_m^{(i)})^ 2$ is small




### Optimising the algorithm for large system: Retroeval & Ranking 


**Retrieval:**

- Generate large list of plausible item candiates
    examples:
      - For each of the last 10 movies watched by the user, find the 10 must similar  movies
      - For the most viewed 3 genres, find the top 10 movies
      - Top 20 movies in the country

- Combine retrieved items into list, removing duplicates and items already watched/purchased.

**Ranking**

- Take the list retrieved and rank using learned model
- Display ranked items to the user

<img src="./screenshots/Retrieveal&Ranking.png" width="500">

## TF implementation

```python

user_NN = tf.keras.models.Sequential(
    [
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu')
        tf.keras.layers.Dense(32)
    ]
)


item_NN = tf.keras.models.Sequential(
    [
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu')
        tf.keras.layers.Dense(32)
    ]
)

# create user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)


# create item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features))
vm = item_NN(input_user)
vm = tf.linalg.l2_normalize(vm, axis=1)


# measure the similarity of the two vector outputs
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and outputs of the model
model = Model([input_user, input_item], output)

# specify the cost function
cost_fn = tf.keras.losses.MeanSquaredError()


```